# アウトバウンド認証

アウトバウンド認証により、エージェントと AgentCore Gateway は、インバウンド認証で認証および認可されたユーザーに代わって、AWS リソースとサードパーティサービスに安全にアクセスできます。AWS リソースまたはサードパーティサービスとの認可を統合するには、インバウンド認証とアウトバウンド認証の両方を設定する必要があります。

AgentCore Identity がサポートするジャストイナフアクセスと安全な権限委任により、エージェントは GitHub、Google、Salesforce、Slack などの AWS リソースとサードパーティツールにシームレスかつ安全にアクセスできます。エージェントは、事前に認可されたユーザーの同意があれば、ユーザーに代わって、または独立してこれらのサービスでアクションを実行できます。さらに、安全なトークンボールトを使用して同意疲れを軽減し、合理化された AI エージェントエクスペリエンスを作成できます。

## アウトバウンド認証の設定

まず、クライアントアプリケーションをサードパーティプロバイダーに登録し、アウトバウンド認証を作成します。AWS リソース、サードパーティサービス、または AgentCore Gateway ターゲットへのアクセスを検証する方法を指定します。OAuth 2LO/3LO または API キーを使用できます。OAuth では、AgentCore Identity が提供するプロバイダーから選択できます。その場合、AgentCore Identity からプロバイダーの設定詳細を入力します。または、カスタムプロバイダーの詳細を提供することもできます。

ユーザーが AWS リソース、サードパーティサービス、または AgentCore Gateway ターゲットへのアクセスを希望する場合、アウトバウンド認証はインカミング認証によって提供されたアクセストークンが有効であることを確認し、有効であればリソースへのアクセスを許可します。

<div style="text-align:center">
    <img src="images/outbound_auth.png" width="90%"/>
</div>


以下は @require_access_token デコレータで使用できる各種パラメータです。


| パラメータ名         | 説明                                                                     |
|:--------------------|:------------------------------------------------------------------------|
| provider_name       | 資格情報プロバイダー名                                                    |
| into                | トークンを注入するパラメータ名                                            |
| scopes              | 要求する OAuth2 スコープ                                                  |
| on_auth_url         | 認可 URL を処理するコールバック                                           |
| auth_flow           | 認証フロータイプ（"M2M" または "USER_FEDERATION"）                        |
| callback_url        | OAuth2 コールバック URL                                                   |
| force_authentication| 再認証を強制                                                             |
| token_poller        | カスタムトークンポーラー実装                                              |

		


# Amazon Bedrock AgentCore Runtime での Strands Agents のホスティング

## 概要


このチュートリアルでは、Strands エージェントを使用して、ユーザーの Google カレンダーからイベントを一覧表示できるスケジューリングエージェントを開発します。Google との資格情報管理を支援する資格情報プロバイダーを設定します。Google の名前付きプロバイダーを使用し、エージェントコードを変更して資格情報プロバイダーを呼び出し、access_token を使用してユーザーのカレンダーイベントまたはスケジュールを Google から取得できます。

### チュートリアルアーキテクチャ

<div style="text-align:center">
    <img src="images/outbound_auth_3lo.png" width="90%"/>
</div>


### チュートリアル詳細

| 情報                | 詳細                                                                     |
|:--------------------|:------------------------------------------------------------------------|
| チュートリアルタイプ | 対話型                                                                   |
| エージェントタイプ   | シングル                                                                 |
| エージェントフレームワーク | Strands Agents                                                     |
| LLM モデル          | Anthropic Claude Haiku 4.5                                               |
| チュートリアルコンポーネント | AgentCore Runtime でのエージェントホスティング。Strands Agent と Claude Model の使用 |
| チュートリアル分野   | クロスバーティカル                                                       |
| 難易度              | 中級                                                                     |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK および boto3                           |
| 資格情報プロバイダー | タイプ：OAuth2 - Google プロバイダー                                      |


### チュートリアルの主な機能

* Amazon Bedrock AgentCore Runtime でのエージェントホスティング
* Claude モデルの使用
* Strands Agents の使用
* OAuth2 Google 資格情報プロバイダーを使用した AgentCore エグレス認証の使用

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* AWS 資格情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker が実行中であること

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Cognito を IDP としてインバウンド認証を設定
アプリクライアントと1人のテストユーザーを持つ Cognito ユーザープールをプロビジョニングしましょう。デプロイされた MCP サーバーにアクセスするための JWT トークンを提供するために Amazon Cognito を使用します。そのために、`utils` スクリプトの `setup_cognito_user_pool` サポート関数を使用します。

注意：Cognito access_token は2時間のみ有効です。access_token の有効期限が切れた場合は、`reauthenticate_user` メソッドを使用して別の access_token を取得できます。

In [ ]:
import sys
import os
import subprocess
from boto3.session import Session

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])
from utils import setup_cognito_user_pool, reauthenticate_user

boto_session = Session()
region = boto_session.region_name
print(f"Region: {region}")

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool("Cognito_3LO_Google")
print("Cognito setup completed ✓")

## OAuth2 用に Google を設定（ユーザーに代わって動作するフロー）
以下の手順に従って、アプリを登録し、プロジェクトを作成し、Google カレンダーアクセス用の OAuth 資格情報を設定します：

このセクションでは、読み取り専用権限で Google Calendar API にアクセスするように Google を設定します。

1. Google Developer Console でプロジェクトを作成
    1.    [Google Developer Console](https://console.developers.google.com/) にアクセス
    2.    上部のナビゲーションバーで「プロジェクトを作成」をクリック
    3.    プロジェクト名を入力
    4.    該当する場合は組織を選択、該当しない場合は「組織なし」のまま
    5.    「作成」をクリック。新しいプロジェクトがプロジェクトリストに表示されます
2. Google Calendar API を有効化
    1.    プロジェクトを選択（チェックボックスを使用）し、左メニュー（ハンバーガーメニュー）を開いて「API とサービス」>「ライブラリ」に移動
    2.    検索バーに「Google Calendar API」と入力
    3.    結果の「Google Calendar API」をクリックし、「有効にする」をクリック
3. OAuth 同意画面を設定
    1.    左メニューで「API とサービス」>「OAuth 同意画面」に移動
    2.    「はじめに」をクリック
    3.    必須フィールドを入力：
    4.    アプリ名
    5.    ユーザーサポートメール
    6. 「次へ」をクリックし、適切なオーディエンスタイプ（内部または外部）を選択、「次へ」をクリック（外部を選択した場合、テストできるユーザーのメール ID を入力してください）
    7.    開発者連絡先情報にメール ID を入力
    8.  利用規約に同意後、「完了」をクリックし、「作成」をクリック
4. テストユーザーとして Google メール ID を追加
    1.    左メニューで「API とサービス」>「OAuth 同意画面」に移動
    2.    左側メニューから「オーディエンス」を選択
    3.    「テストユーザー」の下で「+ ユーザーを追加」をクリックし、Google アカウントの gmail ID を追加
5. OAuth 2.0 資格情報を作成
    1.    左メニューから「API とサービス」>「認証情報」に移動
    2.    「認証情報を作成」>「OAuth クライアント ID」をクリック
    3.    アプリケーションタイプとして「ウェブアプリケーション」を選択
    4.    資格情報の名前を入力
    5.    「作成」をクリック
6. クライアント ID とクライアントシークレットを取得
    1.    作成後、ダイアログにクライアント ID とクライアントシークレットが表示されます。後で使用するためにコピーしてください
    2.    資格情報を JSON ファイルとしてダウンロードするか、アプリの設定で使用するためにコピー
7. データアクセスを更新
    1. 左メニューから「API とサービス」>「認証情報」に移動
    2. ステップ 4d で作成した「ウェブアプリ」を選択
    3. 左側メニューで「データアクセス」を選択
    4. 「スコープを追加または削除」をクリック
    5. ユースケースに基づいてスコープを追加。例：Google カレンダーの場合、「スコープを手動で追加」の下に「https://www.googleapis.com/auth/calendar.readonly」を追加し、「更新」をクリック、その後「保存/更新」をクリック
    6. 「データアクセス」ページでもう一度「保存」をクリックして設定を保存
8. エージェントで資格情報を使用
    1.    次のセクションで、3レッグ OAuth フロー用のクライアント ID、クライアントシークレット、リダイレクト URI を使用するリソース資格情報プロバイダーを設定します

## OAuth2 認可 URL セッションバインディングプロセス

OAuth2 認可 URL セッションバインディングプロセスは、OAuth2 認可セッションが AgentCore Identity で認証されたユーザーと適切に関連付けられることを保証する重要なセキュリティメカニズムです。このプロセスはセッションハイジャックを防ぎ、OAuth トークンが意図したユーザーにのみ付与されることを保証します。

参照：https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html

### セッションバインディングの仕組み
<div style="text-align:center">
    <img src="images/identity-session-binding.png" width="90%"/>
</div>

1. エージェントを呼び出す - エージェントコードが GetResourceOauth2Token API を呼び出して認可 URL を取得します。これは、元のエージェントユーザーが自分が所有するアプリケーションまたはリソースにアクセスしたい場合に行われます。

2. 認可 URL を生成 - AgentCore Identity がユーザーがナビゲートして同意するための認可 URL とセッション URI を生成します。

3. 認可してアクセストークンを取得 - ユーザーが認可 URL にナビゲートし、エージェントがリソースにアクセスすることに同意します。その後、AgentCore Identity は認可リクエストの元のユーザーに関する情報を含めて、ユーザーのブラウザを HTTPS アプリケーションエンドポイントにリダイレクトします。この時点で、HTTPS アプリケーションエンドポイントは、元のエージェントユーザーがアプリケーションの現在ログインしているユーザーと同じかどうかを判断します。一致する場合、アプリケーションエンドポイントは CompleteResourceTokenAuth を呼び出して、AgentCore Identity がアクセストークンを取得して保存できるようにします。

4. エージェントを再呼び出してアクセストークンを取得 - アプリケーションが有効なレスポンスを返すと、エージェントアプリケーションはユーザーに対して元々リクエストされた OAuth2.0 アクセストークンを取得できるようになります。ユーザーが一致しない場合、アプリケーションは何もしないか、試行をログに記録します。

アプリケーションエンドポイントがユーザー ID を検証できるようにすることで、AgentCore Identity はエージェントアプリケーションが常に認可リクエストを開始したユーザーとアクセスに同意したユーザーが同じであることを保証できます。

### このサンプルでの OAuth2 セッションバインディングフローの概要

OAuth2 セッションバインディングプロセスには、アプリケーション、AgentCore Identity、外部 OAuth プロバイダー（Google、Github など）、ローカルコールバックサーバー間で調整するいくつかの重要なステップが含まれます：

#### ステップ 1：アプリケーションコールバック URL を作成
- アプリケーションに公開アクセス可能な HTTPS コールバックエンドポイントを作成
- このエンドポイントは OAuth リダイレクトを処理し、ユーザーセッションを検証します
- 例：`https://myagentapp.com/callback`

#### ステップ 2：コールバック URL でワークロード ID を更新
- コールバック URL をワークロード ID の `AllowedResourceOauth2ReturnUrl` として登録
- これは `UpdateWorkloadIdentity` API を使用して実行されます
- **このチュートリアルでは**：このステップは、ローカルコールバックサーバー URL でワークロード ID を更新する以下のコードによって自動的に処理されます

#### ステップ 3：OAuth2 資格情報プロバイダーを作成
- 外部 OAuth プロバイダーの詳細（クライアント ID、シークレット）で資格情報プロバイダーを設定
- AgentCore Identity はプロバイダーの一意のコールバック URL を返します
- このコールバック URL を外部 OAuth プロバイダー（例：Google Console）に登録

#### ステップ 4：セッション検証とトークン完了を実装
- コールバックエンドポイントは現在のユーザーのセッションを検証する必要があります
- ユーザー識別子とセッション URI で `CompleteResourceTokenAuth` API を呼び出します
- **このチュートリアルでは**：`oauth2_callback_server.py` がこれを自動的に処理します

#### ステップ 5：OAuth フローの実行
- ユーザーがエージェントとのインタラクションを通じて OAuth フローをトリガー
- ユーザーは認可のために外部プロバイダーにリダイレクト
- プロバイダーはセッション情報とともにコールバックにリダイレクト
- セッションバインディングが完了し、OAuth トークンが利用可能になります

### oauth2_callback_server.py を使用したローカル開発

ローカル開発とテストのために、このチュートリアルでは `oauth2_callback_server.py` を使用します：

1. **ローカル FastAPI サーバーを実行**（`localhost:9090`）
   - ヘルスチェック用の `/ping` エンドポイントを提供
   - ユーザートークンを保存する `/userIdentifier/token` エンドポイントを提供
   - OAuth リダイレクトを処理する `/oauth2/callback` エンドポイントを提供

2. **ユーザートークンストレージを管理**
   - Cognito 認証からのユーザーの JWT トークンを保存
   - OAuth セッションを正しいユーザー ID に関連付け

3. **OAuth コールバック処理**
   - `session_id` パラメータ付きの OAuth リダイレクトを受信
   - `CompleteResourceTokenAuth` を呼び出してセッションをバインド
   - フロー完了前にユーザー ID を検証

4. **セッションセキュリティを提供**
   - OAuth セッションが認証されたユーザーにバインドされることを保証
   - OAuth トークンへの不正アクセスを防止

### ワークロード ID 更新との統合

参照したコードスニペットは、OAuth2 セッションバインディングプロセスの重要なステップを実行します：

```python
if launch_result.agent_id:
    workload_name = launch_result.agent_id
    workload_identity = identity_client.get_workload_identity(name=workload_name)
    allowed_resource_oauth_2_return_urls = workload_identity.get("allowedResourceOauth2ReturnUrls") or []
    oauth2_callback_url = get_oauth2_callback_url()
    print(f"Updating workload {workload_name} with callback url {oauth2_callback_url}")

    updated_workload_identity = identity_client.update_workload_identity(
        name=workload_name,
        allowed_resource_oauth_2_return_urls=[*allowed_resource_oauth_2_return_urls, oauth2_callback_url],
    )
```

このコードは：
1. **エージェントのワークロード ID を取得**：ランタイムデプロイメントからのエージェント ID を使用
2. **現在の許可された URL を取得**：既存の `allowedResourceOauth2ReturnUrls` を取得
3. **ローカルコールバック URL を追加**：許可されるリターン URL として `http://localhost:9090/oauth2/callback` を含める
4. **ワークロード ID を更新**：AgentCore Identity にコールバック URL を登録

この登録は、AgentCore Identity が事前登録された URL にのみ OAuth コールバックをリダイレクトするため、追加のセキュリティ層を提供します。

### セキュリティに関する考慮事項

OAuth2 セッションバインディングプロセスにはいくつかのセキュリティ対策が含まれています：
- **URL 検証**：事前登録されたコールバック URL のみ許可
- **セッション検証**：トークン完了前にユーザーセッションを検証する必要あり
- **ユーザー ID バインディング**：OAuth セッションは認証されたユーザーに明示的にバインド
- **トークン分離**：各ユーザーの OAuth トークンは分離され安全

この包括的なアプローチにより、OAuth2 フローが安全で、マルチユーザー環境で正しいユーザーに適切に帰属することが保証されます。

---

### Google OAuth2 資格情報プロバイダーを設定

以下のコードを変更して、以下を置き換えてください：
1. <your-google-client-id> を上記ステップ 6.1 で記録した「クライアント ID」に
2. <your-google-client-secret> を上記ステップ 6.2 で記録した「クライアントシークレット」に

重要：「<」と「>」を含むプレースホルダー文字列全体を置き換えてください


クライアント ID とクライアントシークレットを更新したら、以下のコードを実行して Google 用の資格情報プロバイダーを作成します。
AgentCore Identity のリソース資格情報プロバイダーは、エージェント、ID プロバイダー、リソースサーバー間の複雑な関係を管理するインテリジェントな仲介者として機能します。各プロバイダーは、特定のサービスまたは ID システムに必要な特定のエンドポイント設定をカプセル化します。このサービスは、Google、GitHub、Slack、Salesforce などの人気サービス用の組み込みプロバイダーを提供し、認可サーバーエンドポイントとプロバイダー固有のパラメータが事前設定されているため、開発作業が軽減されます。AgentCore Identity は、OAuth2 互換の任意のリソースサーバーで動作するようにカスタマイズできる設定可能な OAuth2 資格情報プロバイダーを通じてカスタム設定をサポートします。

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient

identity_client = IdentityClient(region)

# Configure Google OAuth2 provider - On-Behalf-Of User
google_provider = identity_client.create_oauth2_credential_provider({
    "name": "google-cal-provider",
    "credentialProviderVendor": "GoogleOauth2",
    "oauth2ProviderConfigInput": {
        "googleOauth2ProviderConfig": {
            "clientId": "<your-google-client-id>",
            "clientSecret": "<your-google-client-secret>"
        }
    }
})
print(google_provider)
print("\n")
print(f"callbackUrl: {google_provider['callbackUrl']}")

## Google/OAuth 2.0 クライアントでコールバック URL を更新
[Google Developer Console](https://console.developers.google.com/) に戻ります

1. Google Developer Console でプロジェクトを選択
    1.    先ほど作成したプロジェクトを選択
2. コールバック URI を更新
    1.    左メニューから「API とサービス」>「認証情報」に移動
    2.    「OAuth 2.0 クライアント ID」の下のクライアントをクリック
    3.    「承認済みのリダイレクト URI」の下に、前のステップのコールバック URL を入力。コールバック URL は印刷されているので、前のステップから簡単にコピーできます
    4.    「保存」をクリック

## AgentCore Runtime へのデプロイ用にエージェントを準備

### Amazon Bedrock でホストされたモデルを使用した Strands Agent
以下は、次の内容を含む Strands エージェントコードです：
1. 今日の Google カレンダーからイベントを取得する「get_calendar_events_today」という新しいツールを作成
2. 前のステップで作成した資格情報プロバイダーを使用して Google から access_token を取得。これには、3LO フローの一部としてユーザーに同意を送信するユーザー同意フローが含まれます
3. Strands エージェントは、ユーザーのアジェンダに関連するユーザーリクエストに対してツールを呼び出します

In [ ]:
# Get the OAuth2 callback URL based on the current environment (notebook/SageMaker)
# This is evaluated HERE in the notebook, not in the agent container
from oauth2_callback_server import get_oauth2_callback_url
oauth2_callback_url_for_agent = get_oauth2_callback_url()

print(f"Callback URL for agent (determined from notebook environment): {oauth2_callback_url_for_agent}")

# Write strands_claude_google_3lo.py with the callback URL embedded as a string literal
google_agent_code = f'''import os
import datetime
import json
import asyncio
import traceback
from typing import Dict, Any, Optional, AsyncGenerator

from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_access_token

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Environment configuration
os.environ["STRANDS_OTEL_ENABLE_CONSOLE_EXPORT"] = "true"
os.environ["OTEL_PYTHON_EXCLUDED_URLS"] = "/ping,/invocations"

# Required OAuth2 scope for Google Calendar API
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

# Global variable to store the access token
google_access_token: Optional[str] = None


@tool(
    name="Get_calendar_events_today",
    description="Retrieves the calendar events for the day from your Google Calendar"
)
def get_calendar_events_today() -> str:
    """
    Retrieve calendar events for today from Google Calendar.
    
    Returns:
        str: JSON string containing events or error information
    """
    global google_access_token
    
    # Check if we already have a token
    if not google_access_token:
        return json.dumps({{
            "auth_required": True,
            "message": "Google Calendar authentication is required. Please wait while we set up the authorization.",
            "events": []
        }})
    
    # Create credentials from the provided access token
    creds = Credentials(token=google_access_token, scopes=SCOPES)
    try:
        service = build("calendar", "v3", credentials=creds)
        
        # Calculate today's time range
        today_start = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        today_end = today_start.replace(hour=23, minute=59, second=59)
        
        # Format with CDT timezone (-05:00)
        time_min = today_start.strftime('%Y-%m-%dT00:00:00-05:00')
        time_max = today_end.strftime('%Y-%m-%dT23:59:59-05:00')
        
        events_result = (
            service.events()
            .list(
                calendarId="primary",
                timeMin=time_min,
                timeMax=time_max,
                singleEvents=True,
                orderBy="startTime",
            )
            .execute()
        )
        events = events_result.get("items", [])

        if not events:
            return json.dumps({{"events": []}})

        return json.dumps({{"events": events}})
        
    except HttpError as error:
        error_message = str(error)
        return json.dumps({{"error": error_message, "events": []}})
    except Exception as e:
        error_message = str(e)
        return json.dumps({{"error": error_message, "events": []}})


# Initialize the agent with tools and your preferred model choice
agent = Agent(
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    tools=[get_calendar_events_today]
)

# Initialize app and streaming queue
app = BedrockAgentCoreApp()


class StreamingQueue:
    """A queue for streaming responses asynchronously."""
    
    def __init__(self):
        self.finished = False
        self.queue = asyncio.Queue()
        
    async def put(self, item: str) -> None:
        """Add an item to the queue."""
        await self.queue.put(item)

    async def finish(self) -> None:
        """Mark the queue as finished."""
        self.finished = True
        await self.queue.put(None)

    async def stream(self) -> AsyncGenerator[str, None]:
        """Stream items from the queue."""
        while True:
            item = await self.queue.get()
            if item is None and self.finished:
                break
            yield item


queue = StreamingQueue()


async def on_auth_url(url: str) -> None:
    """Handle authorization URL callback."""
    print(f"Authorization url: {{url}}")
    await queue.put(f"Authorization url: {{url}}")


async def agent_task(user_message: str) -> None:
    """
    Execute the agent task with authentication handling.
    
    Args:
        user_message: The user's input message
    """
    try:
        await queue.put("Begin agent execution")
        
        # Call the agent first to see if it needs authentication
        response = agent(user_message)
        
        # Extract text content from the response structure
        response_text = ""
        if isinstance(response.message, dict):
            content = response.message.get('content', [])
            if isinstance(content, list):
                for item in content:
                    if isinstance(item, dict) and 'text' in item:
                        response_text += item['text']
        else:
            response_text = str(response.message)
        
        # Check if the response indicates authentication is required
        auth_keywords = [
            "authentication", "authorize", "authorization", "auth", 
            "sign in", "login", "access", "permission", "credential",
            "need authentication", "requires authentication"
        ]
        needs_auth = any(keyword.lower() in response_text.lower() for keyword in auth_keywords)
       
        if needs_auth:
            await queue.put("Authentication required for Google Calendar access. Starting authorization flow...")
            
            # Trigger the 3LO authentication flow
            try:
                global google_access_token
                google_access_token = await need_token_3lo_async(access_token='')
                await queue.put("Authentication successful! Retrying calendar request...")
                
                # Retry the agent call now that we have authentication
                response = agent(user_message)
            except Exception as auth_error:
                print(f"auth_error: {{auth_error}}")
                await queue.put(f"Authentication failed: {{str(auth_error)}}")
        
        await queue.put(response.message)
        await queue.put("End agent execution")
    except Exception as e:
        await queue.put(f"Error: {{str(e)}}")
    finally:
        await queue.finish()


@requires_access_token(
    provider_name="google-cal-provider",
    scopes=SCOPES,
    auth_flow='USER_FEDERATION',
    on_auth_url=on_auth_url,
    force_authentication=True,
    callback_url="{oauth2_callback_url_for_agent}"  # ← Callback URL determined from notebook environment
)
async def need_token_3lo_async(*, access_token: str) -> str:
    """
    Handle 3-legged OAuth token retrieval.
    
    Args:
        access_token: The OAuth access token
        
    Returns:
        str: The access token
    """
    global google_access_token
    google_access_token = access_token
    return access_token


@app.entrypoint
async def agent_invocation(payload: Dict[str, Any]) -> AsyncGenerator[str, None]:
    """
    Main entrypoint for agent invocations.
    
    Args:
        payload: The request payload containing the prompt
        
    Yields:
        str: Streaming response items
    """
    user_message = payload.get(
        "prompt", 
        "No prompt found in input, please guide customer to create a json payload with prompt key"
    )
    
    # Create and start the agent task
    task = asyncio.create_task(agent_task(user_message))
    
    # Return the stream, but ensure the task runs concurrently
    async def stream_with_task() -> AsyncGenerator[str, None]:
        # Stream results as they come
        async for item in queue.stream():
            yield item
        
        # Ensure the task completes
        await task
    
    return stream_with_task()


if __name__ == "__main__":
    app.run()
'''

# Write the file
with open("strands_claude_google_3lo.py", "w") as f:
    f.write(google_agent_code)

print("✅ strands_claude_google_3lo.py written successfully with embedded callback URL")

## エージェントを AgentCore Runtime にデプロイ
CreateAgentRuntime オペレーションは包括的な設定オプションをサポートしており、コンテナイメージ、環境変数、暗号化設定を指定できます。また、クライアントがエージェントと通信する方法を制御するためのプロトコル設定（HTTP、MCP）と認可メカニズムも設定できます。

注意：運用のベストプラクティスは、コードをコンテナとしてパッケージ化し、CI/CD パイプラインと IaC を使用して ECR にプッシュすることです

このチュートリアルでは、Amazon Bedrock AgentCode Python SDK を使用してアーティファクトを簡単にパッケージ化し、AgentCore Runtime にデプロイします。

### AgentCore Runtime デプロイメントの設定

次に、スターターツールキットを使用して、エントリポイント、作成した実行ロール、requirements ファイルで AgentCore Runtime デプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて Dockerfile が生成されます

注意：authorizer_configuration は Cognito でのインバウンド認証用に設定されています。

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

boto_session = Session()
region = boto_session.region_name
print(f"Region: {region}")

discovery_url = cognito_config.get("discovery_url")
client_id = cognito_config.get("client_id")
agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_google_3lo.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    memory_mode="NO_MEMORY",
    agent_name="strands_agent_google_3lo",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id]
        }
    }
)
print(response)

## AgentCore 設定の確認

In [ ]:
!cat .bedrock_agentcore.yaml

### AgentCore Runtime へのエージェントの起動

Dockerfile ができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
from oauth2_callback_server import get_oauth2_callback_url

# Deploy the agent to AgentCore Runtime and get deployment details
launch_result = agentcore_runtime.launch()
print(launch_result)

if launch_result.agent_id:
    # Extract the workload name from the deployed agent's ID for identity management
    workload_name = launch_result.agent_id
    # Retrieve the current workload identity configuration from AgentCore Identity
    workload_identity = identity_client.get_workload_identity(name=workload_name)
    # Extract existing OAuth2 callback URLs that are already registered for this workload
    allowed_resource_oauth_2_return_urls = workload_identity.get("allowedResourceOauth2ReturnUrls") or []
    # Get the local OAuth2 callback server URL for session binding (localhost:9090/oauth2/callback)
    oauth2_callback_url = get_oauth2_callback_url()
    print(f"Updating workload {workload_name} with callback url {oauth2_callback_url}")

    # Register the local callback URL with the workload identity to enable OAuth2 session binding
    updated_workload_identity = identity_client.update_workload_identity(
        name=workload_name,
        allowed_resource_oauth_2_return_urls=[*allowed_resource_oauth_2_return_urls, oauth2_callback_url],
    )
    print(updated_workload_identity)

#### 自動作成されたロールに追加の必要なポリシーを追加

このモデルに初めてアクセスする新しいアカウントでこれを使用している場合、エージェントがモデルにアクセスできるように、自動作成されたロールに追加の必要なポリシーを追加する必要があります。

In [ ]:
import json
import boto3
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

runtime_response = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)
runtime_role = runtime_response['roleArn']

policies_to_add = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "BedrockModelAccess",
            "Effect": "Allow",
            "Action": [
                "aws-marketplace:ViewSubscriptions",
                "aws-marketplace:Subscribe"
            ],
            "Resource": "*"
        }
    ]
}
iam_client = boto3.client(
    'iam',
    region_name=region
)

response = iam_client.put_role_policy(
    PolicyDocument=json.dumps(policies_to_add),
    PolicyName="outbound_policies",
    RoleName=runtime_role.split("/")[1],
)

### AgentCore Runtime ステータスの確認
AgentCore Runtime をデプロイしたので、デプロイメントステータスを確認しましょう

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
print(f"Final status: {status}")

### AgentCore Runtime の呼び出し

最後に、ペイロードで AgentCore Runtime を呼び出すことができます

エージェントが「Get_calendar_events_today」ツールを呼び出し、3レッグ OAuth フローをトリガーすることに気付くでしょう。認可 URL が表示されます。認可 URL をクリックするか、新しいブラウザセッション/タブにコピー/ペーストして、ユーザー同意フローを完了してください。
認可が完了すると、資格情報プロバイダー「google-cal-provider」が Google から access_token を取得し、カレンダーからイベントを取得するツール実行を完了します。

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
from oauth2_callback_server import store_token_in_oauth2_callback_server, wait_for_oauth2_server_to_be_ready

bearer_token = reauthenticate_user(cognito_config.get("client_id"))

oauth2_callback_server_cmd = [sys.executable, "oauth2_callback_server.py", "--region", region]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

try:
    # Start the OAuth2 callback server
    successfully_started_oauth2_server = wait_for_oauth2_server_to_be_ready()
    if not successfully_started_oauth2_server:
        print("Failed to start OAuth2 callback server to handle session binding "
              "(https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html)")
    else:
        store_token_in_oauth2_callback_server(bearer_token)
        invoke_response = agentcore_runtime.invoke(
            {"prompt": "What is in my agenda for today? Highlight the main events!"},
            bearer_token=bearer_token
        )
        print(invoke_response)
finally:
    oauth2_callback_server_process.terminate()

## オプション - Streamlit アプリを使用してエージェントをテスト

素敵なチャットインターフェースを提供する Streamlit ウェブアプリケーションを使用して、デプロイしたエージェントをテストできます。このディレクトリの `chatbot_app_cognito.py` ファイルは、次の機能を持つウェブベースのチャットボットを作成します：

- `.bedrock_agentcore.yaml` から設定を自動的に読み込み
- Cognito 認証を提供
- ストリーミングレスポンス付きのモダンなチャットインターフェースを表示
- Google カレンダーアクセス用の 3LO OAuth フローを処理

### Streamlit アプリの実行

Streamlit アプリはいくつかの方法で実行できます：

#### オプション 1：Jupyter Notebook から実行（現在のディレクトリ）
- 以下のセルを実行して、このノートブックから直接 Streamlit アプリを起動します：
- ログイン：testuser / MyPassword123! の資格情報を使用（Cognito セットアップで作成されたデフォルトのテストユーザー）
- 「ジョークを言って」などのシンプルなプロンプトをテスト
- 「今日のアジェンダは何ですか？」など `Get_calendar_events_today` ツールをトリガーするプロンプトをテスト
- 認可 URL が返されるのが表示されます。URL をクリックするか、新しいブラウザタブ/ウィンドウに URL をコピー/ペーストして、ユーザー同意フローを完了してください

In [ ]:
from chatbot_app_cognito import get_streamlit_url

# Change to the current directory where the chatbot_app_cognito.py file is located
notebook_dir = os.getcwd()

# Start the Streamlit app
print("Starting Streamlit app...")

oauth2_callback_server_cmd = [sys.executable, "oauth2_callback_server.py", "--region", region]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

try:
    wait_for_oauth2_server_to_be_ready()

    # Run streamlit in the current directory
    process = subprocess.Popen([
        sys.executable, "-m", "streamlit", "run", "chatbot_app_cognito.py",
        "--server.port=8501", "--server.showEmailPrompt=false"
    ],
        cwd=notebook_dir,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    # Print the output as it comes
    for line in iter(process.stdout.readline, ''):
        if line:
            if "8501" in line:
                print("\n🎉 Streamlit app is ready!")
                streamlit_url = get_streamlit_url()
                print(f'\n🚀 Streamlit Application URL:\n{streamlit_url}\n')
                print("⚠️ To stop the app, interrupt the kernel or press Ctrl+C in the terminal")
                break

except KeyboardInterrupt:
    print("\nStreamlit app stopped.")
    oauth2_callback_server_process.terminate()
    process.terminate()
except Exception as e:
    print(f"Error starting Streamlit app: {e}")

#### オプション 2：ターミナルから実行

または、ターミナルから Streamlit アプリを実行することもできます：

```bash
# 現在のディレクトリに移動
cd 01-tutorials/03-AgentCore-identity/05-Outbound_Auth_3lo/

# Streamlit アプリを実行
streamlit run chatbot_app_cognito.py
```

#### Streamlit アプリの使用

1. **ログイン**：`testuser` / `MyPassword123!` の資格情報を使用（Cognito セットアップで作成されたデフォルトのテストユーザー）
2. **チャット**：「今日のアジェンダは何ですか？」や「今日のアジェンダから主要なイベントを強調して」などの質問をする
3. **OAuth フロー**：エージェントが Google カレンダーアクセスを必要とする場合、認可 URL が表示されます - クリックして OAuth フローを完了
4. **機能**：アプリには以下が含まれます：
   - リアルタイムストリーミングレスポンス
   - クリック可能な URL
   - モダンなチャットインターフェース
   - コンテキスト認識
   - 情報豊富なメッセージによるエラー処理

アプリは `.bedrock_agentcore.yaml` ファイルからすべての設定を自動的に読み取るため、デプロイしたばかりの同じエージェントランタイムを使用します。

## クリーンアップ（オプション）

- 作成した AgentCore Runtime をクリーンアップしましょう
- 以下のセルのコメントを解除して実行してください

In [ ]:
# launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
# agentcore_control_client = boto3.client(
#     'bedrock-agentcore-control',
#     region_name=region
# )
# ecr_client = boto3.client(
#     'ecr',
#     region_name=region
    
# )

# runtime_delete_response = agentcore_control_client.delete_agent_runtime(
#     agentRuntimeId=launch_result.agent_id,
    
# )

# response = ecr_client.delete_repository(
#     repositoryName=launch_result.ecr_uri.split('/')[1],
#     force=True
# )

## おめでとうございます！